In [ ]:
import dalmatian as dm
from depmapomics import tracker

In [ ]:
# First make sure new star has been run on all samples
wm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_hg38_RNAseq").disable_hound()
rna_samples = wm.get_samples()
rna_samples

In [ ]:
t = tracker.getTracker()

In [ ]:
# Make sure all rna samples on terra are on the tracker
rna_ids = set(rna_samples.index)
tracker_ids = set(t.index)
rna_ids - tracker_ids

In [ ]:
rna_samples.david_internal_bam_filepath.isna().sum()

In [ ]:
for i in rna_ids:
    fn = "gs://cclebams/rnasq_hg38/" + i + ".Aligned.sortedByCoord.out.bam"
    if t.loc[i, "internal_bam_filepath"] != fn:
        print(i)

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
proteincoding_genes_tpm_logp1 = tc.get(name='expression-d035', version=21, file='proteincoding_genes_tpm_logp1')

In [ ]:
'ACH-001767' in proteincoding_genes_tpm_logp1.index

In [ ]:
for i in rna_ids:
    if t.loc[i, "internal_bam_filepath"] != rna_samples.loc[i, "internal_bam_filepath"]:
        print(i)

In [ ]:
from genepy.utils import helper as h

to_move = []
for i in rna_samples.index:
    david_bam = rna_samples.loc[i, 'david_internal_bam_filepath']
    david_bai = rna_samples.loc[i, 'david_internal_bai_filepath']
    old_bam = rna_samples.loc[i, "internal_bam_filepath"]
    old_bai = rna_samples.loc[i, "internal_bai_filepath"]
    
    to_move.append((david_bam, old_bam))
    to_move.append((david_bai, old_bai))
to_move

In [ ]:
# test on one sample
# h.parrun(['gsutil -u broad-firecloud-ccle mv gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/96d2add9-c915-4d1b-8c67-afe2d3f349ec/star_workflow/39f9cd7c-3ed6-48b8-94b3-e95f9ee4673b/call-star/star_out/CDS-ABH0uZ.Aligned.sortedByCoord.out.bam gs://cclebams/rnasq_hg38/CDS-ABH0uZ.Aligned.sortedByCoord.out.bam'], cores=20)
for (i, v) in to_move[:6]:
    print(i + " moving to " + v)

In [ ]:
h.parrun(['gsutil -u broad-firecloud-ccle mv ' + i + ' ' + v for (i, v) in to_move[6:]], cores=20)

In [ ]:
rna_samples.to_csv('rna_terraws_backup.csv')

In [ ]:
cols_to_delete = ['david_more_resources_rsem_genes',
    'david_more_resources_rsem_isoforms',
    'rsem_genes',
    'rsem_isoforms',
    'star_chimeric_bam_file',
    'star_chimeric_bam_index',
    'star_chimeric_junctions',
    'star_junctions',
    'star_junctions_pass1',
    'star_logs',
    'star_read_counts',
    'star_transcriptome_bam']

In [ ]:
wm.delete_sample_attributes(rna_samples[cols_to_delete], delete_files=True, dry_run=False)

In [ ]:
rna_samples = wm.get_samples()
rna_samples

In [ ]:
renaming_dict = {'david_internal_bai_filepath': 'internal_bai_filepath',
    'david_internal_bam_filepath': 'internal_bam_filepath',
    'david_rsem_genes': 'rsem_genes',
    'david_rsem_isoforms': 'rsem_isoforms',
    'david_star_chimeric_bam_file': 'star_chimeric_bam_file',
    'david_star_chimeric_bam_index': 'star_chimeric_bam_index',
    'david_star_chimeric_junctions': 'star_chimeric_junctions',
    'david_star_junctions': 'star_junctions',
    'david_star_junctions_pass1': 'star_junctions_pass1',
    'david_star_logs': 'star_logs',
    'david_star_read_counts': 'star_read_counts',
    'david_star_transcriptome_bam': 'star_transcriptome_bam',
                }

In [ ]:
rna_samples = rna_samples.rename(columns=renaming_dict)
rna_samples

In [ ]:
rna_samples.columns

In [ ]:
wm.update_sample_attributes(rna_samples)

In [ ]:
rna_samples = wm.get_samples()
rna_samples

In [ ]:
renaming_dict.keys()

In [ ]:
wm.delete_sample_attributes(rna_samples[renaming_dict.keys()], delete_files=False, dry_run=False)

In [ ]:
rna_samples = wm.get_samples()

In [ ]:
# some lines are missing rsem run outputs, rerun on them:
rerun_rsem = rna_samples[rna_samples.rsem_genes.isnull()]
rerun_rsem.index

In [ ]:
wm.update_sample_set(sample_set_id="rerun_rsem", sample_ids=rerun_rsem.index)

In [ ]:
rna_samples.star_chimeric_junctions.isnull().sum()